# Example 3: Move Field and Move Method

[Move field in the refactoring catalog](http://refactoring.com/catalog/moveField.html).

[Move method in the refactoring catalog](http://refactoring.com/catalog/moveMethod.html).

---

Say you want to represent a set of animals that can be someone's pet.

In [111]:
class Animal:
    def __init__(self, *, has_scales=False, lays_eggs=False, drinks_milk=False):
        self.has_scales = has_scales
        self.lays_eggs = lays_eggs
        self.drinks_milk = drinks_milk

class Pet:
    def __init__(self, name, animal, joined_family):
        self.name = name
        self.animal = animal
        self.joined_family = joined_family

    def needs_heat_lamp(self):
        return (
            self.animal.has_scales and
            self.animal.lays_eggs and
            not self.animal.drinks_milk)

In [112]:
import datetime
adoption_date = datetime.datetime.utcnow()

In [113]:
my_pet = Pet('Gregory the Gila Monster', Animal(has_scales=True, lays_eggs=True), adoption_date)
print('%s needs a heat lamp? %s' % (my_pet.name, my_pet.needs_heat_lamp()))

Gregory the Gila Monster needs a heat lamp? True


---

You may decide that the name wasn't quite right. The `Pet` is a bit to judgemental, so let's rename it.

In [114]:
my_pet.name = 'Scalia'
print('%s needs a heat lamp? %s' % (my_pet.name, my_pet.needs_heat_lamp()))

Scalia needs a heat lamp? True


---

Thinking about that, perhaps names are an inherent part of the `Animal`, not part of the `Pet`. Move the property into the `Animal` class, use `@property` to alias the old usage to the new usage with a warning. Use `@property.setter` to handle assignment and also issue a warning.

In [115]:
import warnings

In [116]:
class Animal:
    def __init__(self, name, *, has_scales=False, lays_eggs=False, drinks_milk=False):
        self.name = name
        self.has_scales = has_scales
        self.lays_eggs = lays_eggs
        self.drinks_milk = drinks_milk

class Pet:
    def __init__(self, animal, joined_family):
        self.animal = animal
        self.joined_family = joined_family

    @property
    def name(self):
        warnings.warn('Use the pet.animal.name property directly', DeprecationWarning)
        return self.animal.name

    @name.setter
    def name(self, new_name):
        warnings.warn('Use the pet.animal.name property directly', DeprecationWarning)
        self.animal.name = new_name

    def needs_heat_lamp(self):
        return (
            self.animal.has_scales and
            self.animal.lays_eggs and
            not self.animal.drinks_milk)

In [117]:
my_pet = Pet(Animal('Scalia', has_scales=True, lays_eggs=True), adoption_date)
print('%s needs a heat lamp? %s' % (my_pet.name, my_pet.needs_heat_lamp()))

Scalia needs a heat lamp? True


/Users/bslatkin/projects/pycon2016/lib/python3.5/site-packages/ipykernel/__main__.py:15: DeprecationWarning: Use the pet.animal.name property directly


In [118]:
my_pet.name = 'Gilberto the Gila Monster'  # Still assignable on the outer object

/Users/bslatkin/projects/pycon2016/lib/python3.5/site-packages/ipykernel/__main__.py:20: DeprecationWarning: Use the pet.animal.name property directly


---

The new usage won't return any errors or warnings.

In [119]:
my_pet.animal.name  # Accessible in the inner object without warnings

'Gilberto the Gila Monster'

In [120]:
my_pet.animal.name = 'Gilberto'  # Assignable in the inner object
my_pet.animal.name

'Gilberto'

---

Finally, delete the old usage and make it return an error. The new way will work.

In [124]:
class Animal:
    def __init__(self, name, *, has_scales=False, lays_eggs=False, drinks_milk=False):
        self.name = name
        self.has_scales = has_scales
        self.lays_eggs = lays_eggs
        self.drinks_milk = drinks_milk

class Pet:
    def __init__(self, animal, joined_family):
        self.animal = animal
        self.joined_family = joined_family

    def needs_heat_lamp(self):
        return (
            self.animal.has_scales and
            self.animal.lays_eggs and
            not self.animal.drinks_milk)

In [125]:
animal = Animal('Scalia', has_scales=True, lays_eggs=True)
my_pet = Pet(animal, adoption_date)
print('%s needs a heat lamp? %s' % (animal.name, my_pet.needs_heat_lamp()))

Scalia needs a heat lamp? True


In [126]:
my_pet.name  # Expect an error because the property moved

AttributeError: 'Pet' object has no attribute 'name'

---

Say we want to rename `needs_heat_lamp` to `is_reptile` since that's a more general API anyways. Make the old usage point at the new API on the inner object, and have it issue a warning.

In [127]:
class Animal:
    def __init__(self, name, *, has_scales=False, lays_eggs=False, drinks_milk=False):
        self.name = name
        self.has_scales = has_scales
        self.lays_eggs = lays_eggs
        self.drinks_milk = drinks_milk

    @property
    def is_reptile(self):
        return self.has_scales and self.lays_eggs and not self.drinks_milk

class Pet:
    def __init__(self, animal, joined_family):
        self.animal = animal
        self.joined_family = joined_family

    def needs_heat_lamp(self):
        warnings.warn('Use the animal.is_reptile property directly', DeprecationWarning)
        return self.animal.is_reptile

In [128]:
my_pet = Pet(Animal('Gilberto', has_scales=True, lays_eggs=True), adoption_date)
print('%s needs a heat lamp? %s' % (my_pet.animal.name, my_pet.needs_heat_lamp()))

Gilberto needs a heat lamp? True


/Users/bslatkin/projects/pycon2016/lib/python3.5/site-packages/ipykernel/__main__.py:18: DeprecationWarning: Use the animal.is_reptile property directly


---

Move all callers to the new API, and then remove the old `needs_heat_lamp` method.

In [130]:
class Animal:
    def __init__(self, name, *, has_scales=False, lays_eggs=False, drinks_milk=False):
        self.name = name
        self.has_scales = has_scales
        self.lays_eggs = lays_eggs
        self.drinks_milk = drinks_milk

    @property
    def is_reptile(self):
        return self.has_scales and self.lays_eggs and not self.drinks_milk

class Pet:
    def __init__(self, animal, joined_family):
        self.animal = animal
        self.joined_family = joined_family

In [131]:
animal = Animal('Gilberto', has_scales=True, lays_eggs=True)
my_pet = Pet(animal, adoption_date)
print('%s needs a heat lamp? %s' % (animal.name, animal.is_reptile))

Gilberto needs a heat lamp? True


In [132]:
my_pet.needs_heat_lamp()  # Expect an error

AttributeError: 'Pet' object has no attribute 'needs_heat_lamp'